In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import pandas

from numpy import mean
from numpy import std
from numpy import dstack

from keras.layers import Flatten
from keras.layers import Dropout

from keras.utils import to_categorical
from matplotlib import pyplot


In [2]:
dataframe = pandas.read_csv("ALLfeature.csv", header=None)
dataset = dataframe.values
print(dataset)


[['accmeanx' 'accstdx' 'accmaxx' ... 'accsma' 'gyrosma' 'activity']
 ['-1.92301239894' '0.08970598327798304' '-1.685439567' ...
  '13.92218831748' '0.05290931311599999' 'Exercise']
 ['-1.89956705936' '0.05744322654161099' '-1.797349574' ...
  '13.915354729040002' '0.009526680536' 'Exercise']
 ...
 ['-1.76223887468' '1.4788593308555815' '0.579155463' ...
  '13.128671348579994' '2.18813961348' 'Walkingupstair']
 ['-1.61768404854' '1.7910274717745818' '1.4302321180000002' ...
  '14.156987563180001' '2.4643959640600004' 'Walkingupstair']
 ['-1.1109272277800002' '1.4077288909778327' '1.4302321180000002' ...
  '12.561446300180005' '2.0246994448399995' 'Walkingupstair']]


In [3]:
X = dataset[1:,:-1].astype(float)
Y = dataset[1:,-1]
print(Y)


['Exercise' 'Exercise' 'Exercise' ... 'Walkingupstair' 'Walkingupstair'
 'Walkingupstair']


In [4]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(encoded_Y)

In [5]:
dummy_y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test=train_test_split(X,dummy_y,test_size=0.2,random_state=42)

In [10]:
X_train=dstack(X_train)
X_train.shape

(3421, 1, 50)

In [13]:
X_test=dstack(X_test)
X_test.shape

(856, 1, 50)

In [14]:
y_train.shape
# y_train.shape

(3421, 10)

In [15]:
# y_test=dstack(y_test)
y_test.shape

(856, 10)

In [16]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [17]:
# fit and evaluate a model
def evaluate_model(X_train, y_train, X_test, y_test):
    verbose, epochs, batch_size = 0, 50, 32
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
    model = Sequential()
    model.add(LSTM(10, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _,accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    
    
    return accuracy

In [18]:
# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 
# run an experiment
def run_experiment(repeats=4):
    # load data
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(X_train, y_train, X_test, y_test)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [19]:
run_experiment()

>#1: 94.743
>#2: 94.626
>#3: 93.692
>#4: 94.393
[94.7429895401001, 94.62617039680481, 93.69158744812012, 94.39252614974976]
Accuracy: 94.363% (+/-0.408)
